# DogCatsRedux Kaggle Competition

### 1. Import required modules

In [1]:
# Rather than importing everything manually, we'll make things easy
# and load them all in utils.py, and just import them from there.

# 1. install bcolz (pip install bcolz)
# 2. install theano (pip install theano)
# 3. install keras (pip install keras)
# 4. install tensorflow (pip install tensorflow), (pip install tensorflow-gpu)
# 5. install protoclbuff (pip install protobuf)
%matplotlib inline 
import utils; reload(utils)
from utils import *

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


### 2. Load the data (with augmentation)

In [2]:
base_path = '/home/ubuntu/data/redux/' #workspace directory
#gen = image.ImageDataGenerator(rotation_range=15, width_shift_range=0.1, height_shift_range=0.1, zoom_range=0.1, horizontal_flip=True)
gen=image.ImageDataGenerator()

In [3]:
batch_size=100

In [4]:
batches = get_batches(base_path+'train', gen,shuffle=True, batch_size=batch_size)
# NB: We don't want to augment or shuffle the validation set
val_batches = get_batches(base_path+'valid', shuffle=False, batch_size=batch_size)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [5]:
def onehot(x): return np.array(OneHotEncoder().fit_transform(x.reshape(-1,1)).todense())

In [6]:
val_classes = val_batches.classes
trn_classes = batches.classes
val_labels = onehot(val_classes)
trn_labels = onehot(trn_classes)

#### Check the shape of the labels 

In [7]:
trn_labels.shape

(23000, 2)

In [8]:
val_labels.shape

(2000, 2)

### 3. Load the VGG model

In [9]:
from vgg16 import Vgg16
vgg = Vgg16()
model = vgg.model

### 5. No Dropout + Batch normalization + Data Augmentation

Get the convolution model and the fully connected model, we don't need to retrain conv model, so we get the output of the conv model and use this as input to dense, fully connected model. that way when we retrain the fc model, the input doesn't have to fall through all the conv layers

VGG predicts 1000 classes, but we need only 2 classes(cats/dogs). So pop the last Dense layer and add a new DenseLayer of size 2. Also retrain all dense layers.

In [10]:
vgg.model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  1792        zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 64, 226, 226)  0           convolution2d_1[0][0]            
___________________________________________________________________________________________

Find the last conv layer

In [10]:
layers = vgg.model.layers
last_conv_idx = [index for index,layer in enumerate(layers) 
                     if type(layer) is Convolution2D][-1]

Split conv and fully connected layers

In [11]:
conv_layers = layers[:last_conv_idx+1]
conv_model = Sequential(conv_layers)
# Dense layers - also known as fully connected or 'FC' layers
fc_layers = layers[last_conv_idx+1:]

In [40]:
conv_layers[-1].output_shape[1:]

(512, 14, 14)

A function to make weights half, this is done since the VGG model had dropout of 50% and since we are going to remove it to avoid underfitting

In [12]:
def proc_wgts(layer): return [o/2 for o in layer.get_weights()]

Define the optimization algorithm

In [10]:
#opt = RMSprop(lr=0.1, rho=0.7) #what is rho?
opt = Adam()

Create a model for fully conected layers. the architecture is similar to original vgg fc part

In [8]:
def get_fc_bn_model():
    model = Sequential([
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        #BatchNormalization(mode=2),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        #BatchNormalization(mode=2),
        Dense(2, activation='softmax')
        ])

    #for l1,l2 in zip(model.layers, fc_layers): l1.set_weights(proc_wgts(l2))
    
    #pop last layer and add DenseLayer of 2
    #model.pop()
    #model.add(Dense(2, activation='softmax'))
    return model

In [13]:
fc_bn_model = get_fc_bn_model()

make conv model layers non trainable and add fc_bn model to it

In [76]:
for layer in conv_model.layers: layer.trainable = False
# Look how easy it is to connect two models together!
conv_model.add(fc_bn_model)

In [77]:
conv_model.compile(opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [78]:
conv_model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=2, 
                        validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

Epoch 1/2
23000/23000 [==============================] - 651s - loss: 8.0620 - acc: 0.4994 - val_loss: 7.8898 - val_acc: 0.5105
Epoch 2/2
  300/23000 [..............................] - ETA: 593s - loss: 7.5755 - acc: 0.5300

KeyboardInterrupt: 

Since the accuracy I received for the fully connected batch normalized model was very less, trying the normal vgg model with last layer of Dense(2)

In [11]:
model.pop()
for layer in model.layers: layer.trainable=False
model.add(Dense(2, activation='softmax'))
model.compile(opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [34]:
model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=4,
                    validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

Epoch 1/4
23000/23000 [==============================] - 603s - loss: 0.1285 - acc: 0.9660 - val_loss: 0.0370 - val_acc: 0.9890
Epoch 2/4
23000/23000 [==============================] - 610s - loss: 0.0881 - acc: 0.9773 - val_loss: 0.0377 - val_acc: 0.9880
Epoch 3/4
23000/23000 [==============================] - 609s - loss: 0.0799 - acc: 0.9792 - val_loss: 0.0475 - val_acc: 0.9865
Epoch 4/4
23000/23000 [==============================] - 609s - loss: 0.0827 - acc: 0.9785 - val_loss: 0.0386 - val_acc: 0.9865


vgg model's val accuracy is .9865 where as fc_bn_model had 0.51

Save this model

In [35]:
model.save_weights(base_path+"models/ft1.h5")

## Retrain all the Dense layers
While retraining all the dense layers, it is important to load ft1.h5 which is the weights after finetuning the original vgg model.(Replacing last layer with Dense(2)). This is because we don't need to randomly initialize all the weights in the previous layers while training for weights in the last layer. 

In [51]:
from vgg16 import Vgg16
vgg = Vgg16()
model = vgg.model
model.load_weights(base_path+"models/ft1.h5")
model.pop()
for layer in model.layers: layer.trainable=False
model.add(Dense(2, activation='softmax'))
model.compile(opt, loss='categorical_crossentropy', metrics=['accuracy'])
layers = model.layers
last_conv_idx = [index for index,layer in enumerate(layers) 
                     if type(layer) is Convolution2D][-1]

conv_layers = layers[:last_conv_idx+1]
for layer in layers: layer.trainable = True
for layer in conv_layers: layer.trainable = False

In [53]:
#gen = image.ImageDataGenerator(rotation_range=15, width_shift_range=0.1, height_shift_range=0.1, zoom_range=0.1, horizontal_flip=True)
gen = image.ImageDataGenerator()
batches = get_batches(base_path+'train', gen,shuffle=True, batch_size=batch_size)
# NB: We don't want to augment or shuffle the validation set
val_batches = get_batches(base_path+'valid', shuffle=False, batch_size=batch_size)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [49]:
for layer in model.layers:
    print layer.name,layer.trainable

lambda_8 False
zeropadding2d_92 False
convolution2d_92 False
zeropadding2d_93 False
convolution2d_93 False
maxpooling2d_37 False
zeropadding2d_94 False
convolution2d_94 False
zeropadding2d_95 False
convolution2d_95 False
maxpooling2d_38 False
zeropadding2d_96 False
convolution2d_96 False
zeropadding2d_97 False
convolution2d_97 False
zeropadding2d_98 False
convolution2d_98 False
maxpooling2d_39 False
zeropadding2d_99 False
convolution2d_99 False
zeropadding2d_100 False
convolution2d_100 False
zeropadding2d_101 False
convolution2d_101 False
maxpooling2d_40 False
zeropadding2d_102 False
convolution2d_102 False
zeropadding2d_103 False
convolution2d_103 False
zeropadding2d_104 False
convolution2d_104 False
maxpooling2d_42 True
flatten_13 True
dense_45 True
dropout_25 True
dense_46 True
dropout_26 True
dense_47 True


In [54]:
model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=2, 
                        validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

Epoch 1/2
23000/23000 [==============================] - 599s - loss: 0.1883 - acc: 0.9751 - val_loss: 0.0926 - val_acc: 0.9865
Epoch 2/2
23000/23000 [==============================] - 601s - loss: 0.1738 - acc: 0.9780 - val_loss: 0.0910 - val_acc: 0.9865


In [55]:
model.save_weights(base_path+"models/ft2.h5")

In [56]:
base_path+"models/ft2.h5"

'/home/ubuntu/data/redux/models/ft2.h5'

### Data Augmentation


In [58]:
batch_size=64
from vgg16 import Vgg16
vgg = Vgg16()
model = vgg.model
model.pop()
#for layer in model.layers: layer.trainable=False
model.add(Dense(2, activation='softmax'))
model.load_weights(base_path+"models/ft2.h5")

'''
for i in range(0,7):
    model.pop()

model.add(MaxPooling2D(input_shape=(512, 14, 14)))
#make all the layers non trainable
for layer in model.layers: layer.trainable = False
model.add(Flatten())

model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))

#model.add(BatchNormalization())

model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))

#model.add(BatchNormalization())
model.add(Dense(2, activation='softmax'))
    
model.compile(opt, loss='categorical_crossentropy', metrics=['accuracy'])
'''

"\nfor i in range(0,7):\n    model.pop()\n\nmodel.add(MaxPooling2D(input_shape=(512, 14, 14)))\n#make all the layers non trainable\nfor layer in model.layers: layer.trainable = False\nmodel.add(Flatten())\n\nmodel.add(Dense(4096, activation='relu'))\nmodel.add(Dropout(0.5))\n\n#model.add(BatchNormalization())\n\nmodel.add(Dense(4096, activation='relu'))\nmodel.add(Dropout(0.5))\n\n#model.add(BatchNormalization())\nmodel.add(Dense(2, activation='softmax'))\n    \nmodel.compile(opt, loss='categorical_crossentropy', metrics=['accuracy'])\n"

In [46]:
#gen = image.ImageDataGenerator(rotation_range=15, width_shift_range=0.1, height_shift_range=0.1, zoom_range=0.1, horizontal_flip=True)
gen = image.ImageDataGenerator()
batches = get_batches(base_path+'train', gen,shuffle=True, batch_size=batch_size)
# NB: We don't want to augment or shuffle the validation set
val_batches = get_batches(base_path+'valid', shuffle=False, batch_size=batch_size)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [ ]:
model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=2, 
                        validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

### 7. Plot results

In [ ]:
cm = confusion_matrix(val_classes, preds)
plot_confusion_matrix(cm, {'cat':0, 'dog':1})